# L04 Kata - NLP

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

### Explore data

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.head()

In [ ]:
bs=16

### Load data

In [ ]:
np.random.seed(42)
data_lm = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_by_rand_pct(0.1)
                .label_for_lm()
                .databunch(bs=bs))
data_lm.save('data_lm.pkl');

### Train model

In [ ]:
data_lm = load_data(path, 'data_lm.pkl', bs=bs)

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head');

In [ ]:
learn.load('fit_head');

### Fine tune Language Model

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned');

In [ ]:
learn.load('fine_tuned');

##### Sanity check Language Model

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn.save_encoder('fine_tuned_enc')

### Train Classifier

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
data_clas = (TextList.from_csv(path, 'texts.csv', cols='text', vocab=data_lm.vocab)
                .split_from_df(col=2)
                .label_from_df(cols=0)
                 .databunch(bs=bs))
data_clas.save('data_clas.pkl')

In [ ]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [ ]:
data_clas.show_batch()

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc');

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.save('first')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
learn.load('first');

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
learn.load('second');

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('third')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
learn.load('third');

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.predict("I really loved that movie, it was awesome!")

### Accuracy analysis

### Predict on CPU